In [6]:
import numpy as np
import pandas as pd
from xopt.vocs import VOCS

# Make a proper input file.
quad_ranges = np.array([
    [10.2,12.3],
    [-12.5, -5.7],
    [8.8, 12.2],
    [-4.5, -3.7],
    [11.2, 12.8],
    [-14.5, -13.0],
    [12.2, 14.1],
    [-10.0, -6.8]
])


initial_quad_values = [
    11.0361936,
    -12.4199242,
    11.2858292,
    -4.256414 ,
    11.9899023,
    -13.6483384,
    13.2906356,
    -9.5495203
]

Y ="""
variables:
    QUAD:LI26:201:BCTRL: [0, 1] # we will overwrite these
    QUAD:LI26:301:BCTRL: [0, 1]
    QUAD:LI26:401:BCTRL: [0, 1]
    QUAD:LI26:501:BCTRL: [0, 1]
    QUAD:LI26:601:BCTRL: [0, 1]
    QUAD:LI26:701:BCTRL: [0, 1]
    QUAD:LI26:801:BCTRL: [0, 1]
    QUAD:LI26:901:BCTRL: [0, 1]
objectives:
    GDET:FEE1:241:ENRC: MINIMIZE
constraints:
    TOTAL_SOFT_LOSSES: [GREATER_THAN, -25]
    TOTAL_HARD_LOSSES: [GREATER_THAN, -50]

linked_variables: {}
constants: {}
"""

# create list of inputs and outputs
inputs = []
for i in range(2, 10):
    inputs += [f'QUAD:LI26:{i}01']

outputs = [
    'EM2K0:XGMD:HPS:AvgPulseIntensity',
    'GDET:FEE1:241:ENRC',
    'GDET:FEE1:242:ENRC',
    'CBLM:UNDH:1375:I0_LOSS',
    'CBLM:UNDS:1375:I0_LOSS'
]


vocs = VOCS.from_yaml(Y)

#initial_candidate = dict(zip(sorted(vocs.variables.keys()), initial_quad_values))

for idx, name in enumerate(sorted(vocs.variables.keys())):
    vocs.variables[name] = list(quad_ranges[idx])

#subset = ["QUAD:LI26:801:BCTRL","QUAD:LI26:901:BCTRL"]
#vocs.variables = {ele: list(vocs.variables[ele]) for ele in subset}

print(vocs.dict())

{'variables': {'QUAD:LI26:201:BCTRL': [10.2, 12.3], 'QUAD:LI26:301:BCTRL': [-12.5, -5.7], 'QUAD:LI26:401:BCTRL': [8.8, 12.2], 'QUAD:LI26:501:BCTRL': [-4.5, -3.7], 'QUAD:LI26:601:BCTRL': [11.2, 12.8], 'QUAD:LI26:701:BCTRL': [-14.5, -13.0], 'QUAD:LI26:801:BCTRL': [12.2, 14.1], 'QUAD:LI26:901:BCTRL': [-10.0, -6.8]}, 'constraints': {'TOTAL_SOFT_LOSSES': ['GREATER_THAN', -25.0], 'TOTAL_HARD_LOSSES': ['GREATER_THAN', -50.0]}, 'objectives': {'GDET:FEE1:241:ENRC': 'MINIMIZE'}, 'constants': {}, 'linked_variables': {}}


In [9]:
# create evaluator and generator objects
from xopt.evaluator import Evaluator
from xopt.generators.bayesian import BayesianExplorationGenerator
#from measurements import do_measurement

def dummy(x):
    return 0

evaluator = Evaluator(dummy)
generator = BayesianExplorationGenerator(
    vocs
)
print(vocs)
print(generator.options.dict())
generator.options.n_initial = 5
generator.options.optim.num_restarts=1
generator.options.acq.proximal_lengthscales=[1.5, 1.5]

variables={'QUAD:LI26:201:BCTRL': [10.2, 12.3], 'QUAD:LI26:301:BCTRL': [-12.5, -5.7], 'QUAD:LI26:401:BCTRL': [8.8, 12.2], 'QUAD:LI26:501:BCTRL': [-4.5, -3.7], 'QUAD:LI26:601:BCTRL': [11.2, 12.8], 'QUAD:LI26:701:BCTRL': [-14.5, -13.0], 'QUAD:LI26:801:BCTRL': [12.2, 14.1], 'QUAD:LI26:901:BCTRL': [-10.0, -6.8]} constraints={'TOTAL_SOFT_LOSSES': ['GREATER_THAN', -25.0], 'TOTAL_HARD_LOSSES': ['GREATER_THAN', -50.0]} objectives={'GDET:FEE1:241:ENRC': 'MINIMIZE'} constants={} linked_variables={}
{'optim': {'num_restarts': 5, 'raw_samples': 20, 'sequential': True}, 'acq': {'proximal_lengthscales': None, 'monte_carlo_samples': 512}, 'model': {}, 'n_initial': 3}


In [23]:
# run the optimization
from xopt import Xopt
X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)


In [26]:
# submit initial candidates
import pandas as pd
initial_candidates = pd.DataFrame(
    np.array(initial_quad_values).reshape(1,-1), columns=vocs.variable_names
)
print(
    initial_candidates
)
X.submit_data(initial_candidates)
X.process_futures()

   QUAD:LI26:201:BCTRL  QUAD:LI26:301:BCTRL  QUAD:LI26:401:BCTRL  \
0            11.036194           -12.419924            11.285829   

   QUAD:LI26:501:BCTRL  QUAD:LI26:601:BCTRL  QUAD:LI26:701:BCTRL  \
0            -4.256414            11.989902           -13.648338   

   QUAD:LI26:801:BCTRL  QUAD:LI26:901:BCTRL  
0            13.290636             -9.54952  


0

In [27]:
#X.step()

In [28]:
X.data

,QUAD:LI26:201:BCTRL,QUAD:LI26:301:BCTRL,QUAD:LI26:401:BCTRL,QUAD:LI26:501:BCTRL,QUAD:LI26:601:BCTRL,QUAD:LI26:701:BCTRL,QUAD:LI26:801:BCTRL,QUAD:LI26:901:BCTRL,xopt_error,xopt_error_str
0,11.036194,-12.419924,11.285829,-4.256414,11.989902,-13.648338,13.290636,-9.54952,True,"Traceback (most recent call last):\n File ""c:..."
1,11.036194,-12.419924,11.285829,-4.256414,11.989902,-13.648338,13.290636,-9.54952,True,"Traceback (most recent call last):\n File ""c:..."
